# RAG in action: A smarter customer support chatbot

This Jupyter Notebook demonstrates a simplified Retrieval Augmented Generation (RAG) system, focusing on a customer support chatbot use case. Instead of relying on a pre-trained LLM's static knowledge, our chatbot will "augment" its responses by retrieving relevant information from a small, simulated knowledge base of product documents.

We'll create a few dummy text files to represent our product documentation, then build a simple RAG pipeline to answer user queries using this external knowledge.

## 1. Setting up our simulated knowledge base

First, let's create some dummy text files that our RAG system will "read" to build its knowledge base. These files will represent different parts of our fictional "XYZ Product" documentation.

We'll use Python to write these files directly.

In [1]:
import os

# Define the directory for our simulated documents
DOCS_DIR = "product_docs"
os.makedirs(DOCS_DIR, exist_ok=True)

# Content for our dummy product documents
doc_contents = {
    "product_manual_xyz.txt": """
Product Manual for XYZ Wireless Earbuds

1. Introduction:
   Welcome to your new XYZ Wireless Earbuds! These earbuds offer high-quality audio and a comfortable fit.

2. Charging:
   To charge your earbuds, place them in the charging case. The LED indicator on the case will show battery status. A full charge takes approximately 1.5 hours.

3. Pairing:
   Open the charging case. The earbuds will automatically enter pairing mode. On your device (phone/laptop), go to Bluetooth settings and select "XYZ Earbuds". The earbuds will emit a sound upon successful connection.

4. Controls:
   - Single tap (left/right): Play/Pause
   - Double tap (right): Next song
   - Double tap (left): Previous song
   - Triple tap (left/right): Activate voice assistant

5. Troubleshooting:
   - If earbuds are not connecting, ensure they are charged and try resetting them (see section 6).
   - If audio is distorted, check your device's volume and ensure no interference.

6. Resetting the Earbuds:
   To reset your XYZ Wireless Earbuds to factory settings, place both earbuds in the charging case. Press and hold the button on the charging case for 15 seconds until the LED light blinks red three times. Then, close the case and reopen it.
""",
    "faq_warranty.txt": """
Frequently Asked Questions (FAQ) - XYZ Wireless Earbuds

Q: What is the warranty period for the XYZ Earbuds?
A: The XYZ Wireless Earbuds come with a 1-year limited warranty from the date of purchase. Please retain your proof of purchase for warranty claims.

Q: How do I claim warranty?
A: To claim warranty, please visit our support page at [www.xyztech.com/support](https://www.xyztech.com/support) and fill out the warranty claim form, attaching your receipt. Our team will contact you within 2-3 business days.

Q: Are the XYZ Earbuds waterproof?
A: The XYZ Earbuds are splash-resistant (IPX4 rating), meaning they can withstand light rain and sweat. They are NOT designed for submersion in water.

Q: What devices are compatible?
A: The earbuds are compatible with any Bluetooth-enabled device, including smartphones, tablets, and laptops.
""",
    "troubleshooting_guide.txt": """
Troubleshooting Guide for XYZ Wireless Earbuds

Issue: Earbuds not charging.
Solution: Ensure the charging case has power. Check the charging cable and adapter. Clean the charging contacts on both earbuds and the case with a dry cotton swab. If still not charging, contact support.

Issue: One earbud not working.
Solution:
1. Ensure both earbuds are charged.
2. Try resetting the earbuds (refer to the product manual for instructions).
3. Re-pair the earbuds with your device.
4. If the issue persists, it might be a hardware problem.

Issue: Low volume or distorted sound.
Solution:
1. Check the volume level on your connected device.
2. Ensure there's no obstruction in the earbud's speaker grill.
3. Try playing audio from a different source to rule out the media file.
4. Clean the earbuds.
"""
}

# Write the content to files
for filename, content in doc_contents.items():
    filepath = os.path.join(DOCS_DIR, filename)
    with open(filepath, "w") as f:
        f.write(content.strip()) # .strip() removes leading/trailing whitespace for cleaner files
    print(f"Created: {filepath}")


Created: product_docs\product_manual_xyz.txt
Created: product_docs\faq_warranty.txt
Created: product_docs\troubleshooting_guide.txt


## 2. Core RAG Components

Now, let's define the core components of our RAG system:

* `DocumentChunk`: A simple class to hold text and its embedding.

* `EmbeddingModel`: A placeholder for converting text into numerical vectors (embeddings). In a real application, you'd use a pre-trained model like SentenceTransformers or an API like OpenAI's embeddings.

* `VectorDatabase`: A simplified in-memory "database" to store our document chunks and perform similarity searches. In production, this would be a specialized vector database.

* `simulated_llm`: A stand-in for a Large Language Model that generates responses based on a given prompt.

In [2]:
import numpy as np
from typing import List, Dict, Tuple
import glob # To find our created text files

class DocumentChunk:
    """Represents a chunk of text from our knowledge base."""
    def __init__(self, text: str, source_file: str, chunk_id: str):
        self.text = text
        self.source_file = source_file
        self.chunk_id = chunk_id
        self.embedding: List[float] = [] # To be filled by the embedding model

    def __repr__(self):
        return f"DocumentChunk(id='{self.chunk_id}', source='{self.source_file}', text='{self.text[:50]}...')"

class EmbeddingModel:
    """
    A conceptual embedding model.
    In a real scenario, this would use a pre-trained model to convert text into meaningful vectors.
    For simplicity, we're generating dummy embeddings based on a hash.
    """
    def get_embedding(self, text: str) -> List[float]:
        # Generating a very simple, non-semantic dummy embedding
        # In a real application, use a library like 'sentence-transformers' or a model API
        np.random.seed(hash(text) % (2**32 - 1)) # Seed for reproducibility based on text
        return np.random.rand(768).tolist() # Return a list of 768 random floats

    def cosine_similarity(self, vec1: List[float], vec2: List[float]) -> float:
        """Calculates cosine similarity between two vectors."""
        vec1_np = np.array(vec1)
        vec2_np = np.array(vec2)
        dot_product = np.dot(vec1_np, vec2_np)
        norm_vec1 = np.linalg.norm(vec1_np)
        norm_vec2 = np.linalg.norm(vec2_np)
        if norm_vec1 == 0 or norm_vec2 == 0:
            return 0.0 # Avoid division by zero
        return dot_product / (norm_vec1 * norm_vec2)

class VectorDatabase:
    """
    A highly simplified in-memory vector database.
    In a real system, this would be a dedicated vector database for efficient large-scale search.
    """
    def __init__(self, embedding_model: EmbeddingModel):
        self.store: Dict[str, DocumentChunk] = {}
        self.embedding_model = embedding_model
        self._chunk_counter = 0

    def add_document(self, filepath: str):
        """Reads a file, chunks its content, and adds to the store with embeddings."""
        with open(filepath, 'r') as f:
            full_text = f.read()

        # Simple chunking: split by paragraphs or double newlines
        # For more advanced chunking, consider libraries like LangChain's TextSplitter
        chunks = [c.strip() for c in full_text.split('\n\n') if c.strip()]
        
        print(f"Processing {len(chunks)} chunks from {filepath}...")
        for i, text in enumerate(chunks):
            chunk_id = f"{os.path.basename(filepath).replace('.', '_')}_chunk_{self._chunk_counter}"
            doc_chunk = DocumentChunk(text=text, source_file=filepath, chunk_id=chunk_id)
            doc_chunk.embedding = self.embedding_model.get_embedding(text)
            self.store[chunk_id] = doc_chunk
            self._chunk_counter += 1
        print(f"Finished processing {filepath}.")


    def find_similar_chunks(self, query_embedding: List[float], top_k: int = 3) -> List[Tuple[DocumentChunk, float]]:
        """
        Finds the top_k most similar document chunks to the query embedding.
        Returns a list of (DocumentChunk, similarity_score) tuples.
        """
        if not self.store:
            return []

        similarities = []
        for chunk_id, chunk in self.store.items():
            if chunk.embedding:
                score = self.embedding_model.cosine_similarity(query_embedding, chunk.embedding)
                similarities.append((chunk, score))
        
        # Sort by similarity score in descending order
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_k]

def simulated_llm(prompt: str) -> str:
    """
    A simplified Large Language Model (LLM) stand-in.
    In a real application, this would be an API call to a powerful LLM.
    It tries to give a slightly more "aware" response if relevant keywords are in the prompt context.
    """
    prompt_lower = prompt.lower()

    if "reset" in prompt_lower and "xyz wireless earbuds" in prompt_lower and "button" in prompt_lower:
        return "Based on the provided information, to reset your XYZ Wireless Earbuds, place both earbuds in the charging case, then press and hold the button on the charging case for 15 seconds until the LED light blinks red three times. After that, close the case and reopen it."
    elif "warranty" in prompt_lower and "xyz wireless earbuds" in prompt_lower and "1-year" in prompt_lower:
        return "The XYZ Wireless Earbuds come with a 1-year limited warranty from the date of purchase. Remember to keep your proof of purchase for any warranty claims."
    elif "pairing" in prompt_lower and "bluetooth" in prompt_lower:
        return "According to the manual, to pair your XYZ Earbuds, open the charging case to enter pairing mode. Then, go to your device's Bluetooth settings and select 'XYZ Earbuds'."
    elif "waterproof" in prompt_lower or "submersion" in prompt_lower:
        return "The XYZ Earbuds are splash-resistant (IPX4 rating) and can handle light rain or sweat. However, they are NOT designed for submersion in water."
    elif "low volume" in prompt_lower or "distorted sound" in prompt_lower:
        return "If you're experiencing low volume or distorted sound, check your device's volume, ensure no obstruction in the earbud's speaker grill, and try playing audio from a different source. Cleaning the earbuds might also help."
    else:
        return "I couldn't find specific information about that in my current knowledge base. Could you please rephrase your question or provide more details? You can also visit our website for comprehensive support."



## 3. Ingesting Documents into the Vector Database

Now, let's load the content from our simulated text files into our `VectorDatabase`. Each relevant section will be turned into a `DocumentChunk` and embedded.

In [3]:
# Initialize the embedding model and vector database
embedding_model = EmbeddingModel()
vector_db = VectorDatabase(embedding_model)

# Find all text files in our DOCS_DIR
document_files = glob.glob(os.path.join(DOCS_DIR, "*.txt"))

# Add each document to our vector database
if document_files:
    for doc_file in document_files:
        vector_db.add_document(doc_file)
    print(f"\nKnowledge base built with {len(vector_db.store)} chunks from {len(document_files)} files.")
else:
    print("No document files found. Please ensure files were created correctly.")


Processing 5 chunks from product_docs\faq_warranty.txt...
Finished processing product_docs\faq_warranty.txt.
Processing 7 chunks from product_docs\product_manual_xyz.txt...
Finished processing product_docs\product_manual_xyz.txt.
Processing 4 chunks from product_docs\troubleshooting_guide.txt...
Finished processing product_docs\troubleshooting_guide.txt.

Knowledge base built with 16 chunks from 3 files.


## 4. The RAG Chatbot Function

This function ties everything together:

1. Takes a user query.

2. Embeds the query.

3. Retrieves the most similar document chunks.

4. Constructs an augmented prompt for the LLM.

5. Calls the simulated LLM and returns the response.

In [4]:
def rag_chatbot_query(user_query: str, top_k_chunks: int = 2):
    """
    Main RAG chatbot function.
    Processes a user query by retrieving relevant context and augmenting an LLM prompt.
    """
    print(f"\n--- User Query: {user_query} ---")

    # 1. Embed the user's query
    query_embedding = embedding_model.get_embedding(user_query)
    print("Query embedded.")

    # 2. Retrieve relevant chunks from the vector database
    # Get (chunk, similarity_score) tuples
    relevant_chunks_with_scores = vector_db.find_similar_chunks(query_embedding, top_k=top_k_chunks)
    
    if not relevant_chunks_with_scores:
        print("No relevant chunks found.")
        context = "No specific context found in the knowledge base."
    else:
        print(f"Retrieved {len(relevant_chunks_with_scores)} relevant chunks:")
        for chunk, score in relevant_chunks_with_scores:
            print(f"  - Source: {chunk.source_file}, Score: {score:.4f}, Text: '{chunk.text[:100]}...'")
        
        # 3. Augment the prompt with retrieved context
        context = "\n\n".join([chunk.text for chunk, _ in relevant_chunks_with_scores])

    # Construct the augmented prompt for the LLM
    augmented_prompt = f"""
You are a helpful customer support assistant for XYZ Wireless Earbuds.
Use ONLY the following pieces of context to answer the question.
If the answer is not found in the context, politely state that you don't have enough information.

Context:
{context}

Question: {user_query}

Answer:
"""
    print("\n--- Augmented Prompt for LLM (truncated for display) ---")
    print(augmented_prompt[:500] + "..." if len(augmented_prompt) > 500 else augmented_prompt)

    # 4. Generate the response using the LLM
    response = simulated_llm(augmented_prompt)
    print("\n--- Chatbot's Response ---")
    print(response)
    print("-" * 50)
    return response



## 5. Testing Our RAG Chatbot

Let's ask our RAG chatbot some questions and see how it performs!

In [5]:
# Test queries
rag_chatbot_query("How do I reset my XYZ Wireless Earbuds?")


--- User Query: How do I reset my XYZ Wireless Earbuds? ---
Query embedded.
Retrieved 2 relevant chunks:
  - Source: product_docs\faq_warranty.txt, Score: 0.7631, Text: 'Q: What is the warranty period for the XYZ Earbuds?
A: The XYZ Wireless Earbuds come with a 1-year l...'
  - Source: product_docs\faq_warranty.txt, Score: 0.7561, Text: 'Q: Are the XYZ Earbuds waterproof?
A: The XYZ Earbuds are splash-resistant (IPX4 rating), meaning th...'

--- Augmented Prompt for LLM (truncated for display) ---

You are a helpful customer support assistant for XYZ Wireless Earbuds.
Use ONLY the following pieces of context to answer the question.
If the answer is not found in the context, politely state that you don't have enough information.

Context:
Q: What is the warranty period for the XYZ Earbuds?
A: The XYZ Wireless Earbuds come with a 1-year limited warranty from the date of purchase. Please retain your proof of purchase for warranty claims.

Q: Are the XYZ Earbuds waterproof?
A: The XYZ Ear

'The XYZ Wireless Earbuds come with a 1-year limited warranty from the date of purchase. Remember to keep your proof of purchase for any warranty claims.'

In [6]:
rag_chatbot_query("What's the warranty on these earbuds?")


--- User Query: What's the warranty on these earbuds? ---
Query embedded.
Retrieved 2 relevant chunks:
  - Source: product_docs\faq_warranty.txt, Score: 0.7710, Text: 'Q: What devices are compatible?
A: The earbuds are compatible with any Bluetooth-enabled device, inc...'
  - Source: product_docs\troubleshooting_guide.txt, Score: 0.7686, Text: 'Issue: Low volume or distorted sound.
Solution:
1. Check the volume level on your connected device.
...'

--- Augmented Prompt for LLM (truncated for display) ---

You are a helpful customer support assistant for XYZ Wireless Earbuds.
Use ONLY the following pieces of context to answer the question.
If the answer is not found in the context, politely state that you don't have enough information.

Context:
Q: What devices are compatible?
A: The earbuds are compatible with any Bluetooth-enabled device, including smartphones, tablets, and laptops.

Issue: Low volume or distorted sound.
Solution:
1. Check the volume level on your connected device.
2

"If you're experiencing low volume or distorted sound, check your device's volume, ensure no obstruction in the earbud's speaker grill, and try playing audio from a different source. Cleaning the earbuds might also help."

In [7]:
rag_chatbot_query("Can I take my XYZ Earbuds swimming?")


--- User Query: Can I take my XYZ Earbuds swimming? ---
Query embedded.
Retrieved 2 relevant chunks:
  - Source: product_docs\product_manual_xyz.txt, Score: 0.7618, Text: '6. Resetting the Earbuds:
   To reset your XYZ Wireless Earbuds to factory settings, place both earb...'
  - Source: product_docs\troubleshooting_guide.txt, Score: 0.7582, Text: 'Issue: Earbuds not charging.
Solution: Ensure the charging case has power. Check the charging cable ...'

--- Augmented Prompt for LLM (truncated for display) ---

You are a helpful customer support assistant for XYZ Wireless Earbuds.
Use ONLY the following pieces of context to answer the question.
If the answer is not found in the context, politely state that you don't have enough information.

Context:
6. Resetting the Earbuds:
   To reset your XYZ Wireless Earbuds to factory settings, place both earbuds in the charging case. Press and hold the button on the charging case for 15 seconds until the LED light blinks red three times. Then, cl

'Based on the provided information, to reset your XYZ Wireless Earbuds, place both earbuds in the charging case, then press and hold the button on the charging case for 15 seconds until the LED light blinks red three times. After that, close the case and reopen it.'

In [8]:
rag_chatbot_query("My left earbud is not making any sound. What should I do?")


--- User Query: My left earbud is not making any sound. What should I do? ---
Query embedded.
Retrieved 2 relevant chunks:
  - Source: product_docs\troubleshooting_guide.txt, Score: 0.7687, Text: 'Issue: Earbuds not charging.
Solution: Ensure the charging case has power. Check the charging cable ...'
  - Source: product_docs\product_manual_xyz.txt, Score: 0.7659, Text: 'Product Manual for XYZ Wireless Earbuds...'

--- Augmented Prompt for LLM (truncated for display) ---

You are a helpful customer support assistant for XYZ Wireless Earbuds.
Use ONLY the following pieces of context to answer the question.
If the answer is not found in the context, politely state that you don't have enough information.

Context:
Issue: Earbuds not charging.
Solution: Ensure the charging case has power. Check the charging cable and adapter. Clean the charging contacts on both earbuds and the case with a dry cotton swab. If still not charging, contact support.

Product Manual for X...

--- Chatbot's Respo

"I couldn't find specific information about that in my current knowledge base. Could you please rephrase your question or provide more details? You can also visit our website for comprehensive support."

In [9]:
rag_chatbot_query("Tell me about the history of artificial intelligence.") # Out of scope question


--- User Query: Tell me about the history of artificial intelligence. ---
Query embedded.
Retrieved 2 relevant chunks:
  - Source: product_docs\troubleshooting_guide.txt, Score: 0.7707, Text: 'Issue: Low volume or distorted sound.
Solution:
1. Check the volume level on your connected device.
...'
  - Source: product_docs\faq_warranty.txt, Score: 0.7694, Text: 'Q: What devices are compatible?
A: The earbuds are compatible with any Bluetooth-enabled device, inc...'

--- Augmented Prompt for LLM (truncated for display) ---

You are a helpful customer support assistant for XYZ Wireless Earbuds.
Use ONLY the following pieces of context to answer the question.
If the answer is not found in the context, politely state that you don't have enough information.

Context:
Issue: Low volume or distorted sound.
Solution:
1. Check the volume level on your connected device.
2. Ensure there's no obstruction in the earbud's speaker grill.
3. Try playing audio from a different source to rule out the med

"If you're experiencing low volume or distorted sound, check your device's volume, ensure no obstruction in the earbud's speaker grill, and try playing audio from a different source. Cleaning the earbuds might also help."

In [10]:
rag_chatbot_query("How long does it take to charge the case?")


--- User Query: How long does it take to charge the case? ---
Query embedded.
Retrieved 2 relevant chunks:
  - Source: product_docs\faq_warranty.txt, Score: 0.7669, Text: 'Q: What devices are compatible?
A: The earbuds are compatible with any Bluetooth-enabled device, inc...'
  - Source: product_docs\troubleshooting_guide.txt, Score: 0.7637, Text: 'Issue: Earbuds not charging.
Solution: Ensure the charging case has power. Check the charging cable ...'

--- Augmented Prompt for LLM (truncated for display) ---

You are a helpful customer support assistant for XYZ Wireless Earbuds.
Use ONLY the following pieces of context to answer the question.
If the answer is not found in the context, politely state that you don't have enough information.

Context:
Q: What devices are compatible?
A: The earbuds are compatible with any Bluetooth-enabled device, including smartphones, tablets, and laptops.

Issue: Earbuds not charging.
Solution: Ensure the charging case has power. Check the charging cab

"I couldn't find specific information about that in my current knowledge base. Could you please rephrase your question or provide more details? You can also visit our website for comprehensive support."

## 6. Cleanup (Optional)

You can run this cell to remove the dummy product document files after you're done.

In [11]:
"""
import shutil

if os.path.exists(DOCS_DIR):
    shutil.rmtree(DOCS_DIR)
    print(f"Removed directory: {DOCS_DIR}")
else:
    print(f"Directory {DOCS_DIR} not found, no cleanup needed.")
"""

'\nimport shutil\n\nif os.path.exists(DOCS_DIR):\n    shutil.rmtree(DOCS_DIR)\n    print(f"Removed directory: {DOCS_DIR}")\nelse:\n    print(f"Directory {DOCS_DIR} not found, no cleanup needed.")\n'